In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [0]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [0]:
def createmodel():
    model = Sequential()
    lstm_out=196
    embed_dim=128
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    #print(model.summary())

In [11]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
print(integer_encoded)
print(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [12]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print('score',score)
print('accuracy',acc)
print('Model Metrics Names',model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 19s - loss: 0.8285 - accuracy: 0.6421
Epoch 2/20
 - 17s - loss: 0.6866 - accuracy: 0.7081
Epoch 3/20
 - 17s - loss: 0.6214 - accuracy: 0.7366
Epoch 4/20
 - 17s - loss: 0.5786 - accuracy: 0.7616
Epoch 5/20
 - 17s - loss: 0.5364 - accuracy: 0.7766
Epoch 6/20
 - 17s - loss: 0.4936 - accuracy: 0.8004
Epoch 7/20
 - 17s - loss: 0.4615 - accuracy: 0.8118
Epoch 8/20
 - 17s - loss: 0.4279 - accuracy: 0.8277
Epoch 9/20
 - 17s - loss: 0.3921 - accuracy: 0.8410
Epoch 10/20
 - 18s - loss: 0.3711 - accuracy: 0.8506
Epoch 11/20
 - 17s - loss: 0.3526 - accuracy: 0.8549
Epoch 12/20
 - 18s - loss: 0.3254 - accuracy: 0.8677
Epoch 13/20
 - 18s - loss: 0.3113 - accuracy: 0.8745
Epoch 14/20
 - 17s - loss: 0.3001 - accuracy: 0.8784
Epoch 15/20
 - 17s - loss: 0.2810 - accuracy: 0.8886
Epoch 16/20
 - 17s - loss: 0.2773 - accuracy: 0.8884
Epoch 17/20
 - 17s - loss: 0.2588 - accuracy: 0.8941
Epoch 18/20
 - 17s - loss: 0.2504 - accuracy: 0.8968
Epoch 19/20
 - 17s - loss: 0.2480 - accuracy: 0.8995
Ep

In [0]:
model.save('sample.h5')

In [14]:
from keras.models import load_model
model= load_model('sample.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [15]:
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
sentence = tokenizer.texts_to_sequences(sentence)
#padding the tweet to have exactly the same shape as `embedding_2` input
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 1:
  print("Neutral")
elif sentiment == 0:
  print("Negative")
elif sentiment == 2:
  print("Positive")
else:
  print("Can not be determined")

2
Positive


2. Apply GridSearchCV on the source code provided in the class

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 43s - loss: 0.8343 - accuracy: 0.6391
Epoch 1/1
 - 43s - loss: 0.8261 - accuracy: 0.6500
Epoch 1/1
 - 43s - loss: 0.8303 - accuracy: 0.6433
Epoch 1/1
 - 43s - loss: 0.8337 - accuracy: 0.6420
Epoch 1/1
 - 44s - loss: 0.8236 - accuracy: 0.6438
Epoch 1/2
 - 43s - loss: 0.8305 - accuracy: 0.6434
Epoch 2/2
 - 43s - loss: 0.6884 - accuracy: 0.7112
Epoch 1/2
 - 43s - loss: 0.8297 - accuracy: 0.6458
Epoch 2/2
 - 42s - loss: 0.6950 - accuracy: 0.7042
Epoch 1/2
 - 44s - loss: 0.8265 - accuracy: 0.6411
Epoch 2/2
 - 44s - loss: 0.6873 - accuracy: 0.7113
Epoch 1/2
 - 43s - loss: 0.8291 - accuracy: 0.6453
Epoch 2/2
 - 42s - loss: 0.6812 - accuracy: 0.7120
Epoch 1/2
 - 42s - loss: 0.8217 - accuracy: 0.6468
Epoch 2/2
 - 42s - loss: 0.6766 - accuracy: 0.7099
Epoch 1/1
 - 22s - loss: 0.8374 - accuracy: 0.6407
Epoch 1/1
 - 21s - loss: 0.8367 - accuracy: 0.6400
Epoch 1/1
 - 22s - loss: 0.8392 - accuracy: 0.6369
Epoch 1/1
 - 22s - loss: 0.8345 - accuracy: 0.6416
Epoch 1/1
 - 22s - loss: 0.8332

3. Apply the code on spamdata set availablein thesourcecode (text classification on the spam.csvdata set)

In [0]:
data1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/spam.csv',encoding="ISO-8859-1")

# Keeping only the neccessary columns
data1 = data1[['v1','v2']]

In [0]:
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='sigmoid'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [23]:
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 34s - loss: 0.2200 - accuracy: 0.9279
Epoch 2/5
 - 34s - loss: 0.0453 - accuracy: 0.9858
Epoch 3/5
 - 34s - loss: 0.0242 - accuracy: 0.9925
Epoch 4/5
 - 34s - loss: 0.0143 - accuracy: 0.9965
Epoch 5/5
 - 34s - loss: 0.0193 - accuracy: 0.9952
0.07585310827446427
0.9798803925514221
['loss', 'accuracy']
